In [ ]:
print("------ IMPORT LIBRARY ------")

import warnings
warnings.filterwarnings('ignore')
from nilmtk import DataSet
from rnndisaggregator import RNNDisaggregator
from nilmtk.datastore import HDFDataStore
import matplotlib.pyplot as plt
import metrics

# skip if you already have .h5 dataset
# from nilmtk.dataset_converters import convert_ukdale
# convert_ukdale('./data/REDD/low_freq', './data/REDD/redd.h5') 

In [ ]:
print("------ SET GLOBAL VARIABLES ------")

DATASET = "./data/REDD/redd.h5" # path of redd data
MODEL = "./data/REDD/microwave-model-redd.h5" # path of model
DISAG_FILE = "./data/REDD/microwave-disag-out.h5" # path of disaggregation output
APPLIANCE = 'microwave' # the network will be trained on this appliance
SAMPLE_PERIOD = 1 # sample period

WINDOWS = {
    'train': {
        1: ("2011-04-18", "2011-05-24"),
        2: ("2011-04-17 23:18:00", "2011-05-22 23:59:00"),
        3: ("2011-04-16 05:11:00", "2011-05-31 00:19:00"),
        5: ("2011-04-18 04:24:00", "2011-06-01 00:20:00"),
    },
    'test': {
        6: ("2011-05-21 19:39", "2011-06-14 05:31")
    }
}

# True if the model must be trained
TRAINING = True

In [ ]:
print("------ PREPARING DATA ------")

train_mainlist = []
train_meterlist = []

# for each building
for key in WINDOWS['train'].keys():
    train = DataSet(DATASET)
    train.set_window(start=WINDOWS['train'][key][0], end=WINDOWS['train'][key][1])
    train_elec = train.buildings[key]
    
    train_mainlist.append(train_elec.mains()all_meters()[0])
    train_meterlist.append(train_elec.submeters()[APPLIANCE])

In [ ]:
rnn = RNNDisaggregator()

if TRAINING:
    print("------ START TRAINING ------")
    rnn.train_across_buildings(train_mainlist, train_meterlist, epochs=5, sample_period=SAMPLE_PERIOD)
    rnn.export_model(MODEL)
else:
    print("------ IMPORT MODEL ------")
    rnn.import_model(MODEL)

In [ ]:
print("------ PREPARING TEST DATA ------")
test = DataSet(DATASET)
test.set_window(start=WINDOWS['test'][6][0], end=WINDOWS['test'][6][1])
test_elec = test.buildings[6].elec
test_mains = test_elec.mains()

disag_filename = DISAG_FILE # The filename of the resulting datastore
output = HDFDataStore(disag_filename, 'w')

print("------ START DISAGGREGATION ------")
rnn.disaggregate(test_mains, output, train_meter[0], sample_period=SAMPLE_PERIOD)
output.close()

In [ ]:
result = DataSet(disag_filename)
res_elec = result.buildings[6].elec
predicted = res_elec[APPLIANCE]
ground_truth = test_elec[APPLIANCE]

print("------ PLOTTING PREDICTION AND GROUND TRUTH ------")
predicted.power_series_all_data().plot()
ground_truth.power_series_all_data().plot()
plt.show()

In [ ]:
print("============ Relative error in total energy: {}".format(metrics.relative_error_total_energy(predicted, ground_truth)))
print("============ Mean absolute error(in Watts): {}".format(metrics.mean_absolute_error(predicted, ground_truth)))